In [277]:
import nltk
import gensim
import textblob
import spacy
nlp = spacy.load('en')
from wordcloud import wordcloud
import string
import re
from bs4 import BeautifulSoup

from nltk.corpus import stopwords,wordnet
stopwords_ = stopwords.words('english')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer


pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', -1)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings("ignore")


from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import f1_score,roc_auc_score,precision_score,recall_score,accuracy_score
from sklearn.model_selection import train_test_split,StratifiedKFold

In [278]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [279]:
k = pd.DataFrame()
k["train"] = round(100*train.isna().sum()/len(train),2)
k["test"] = round(100*test.isna().sum()/len(test),2)
k.T

,id,label,tweet
train,0.0,0.0,0.0
test,0.0,NaN,0.0


In [280]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [281]:
test.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me download apps. #ugh #apple sucks
1,7922,currently shitting my fucking pants. #apple #iMac #cashmoney #raddest #swagswagswag http://instagr.am/p/UUIS0bIBZo/
2,7923,"I'd like to puts some CD-ROMS on my iPad, is that possible?' — Yes, but wouldn't that block the screen?\n"
3,7924,"My ipod is officially dead. I lost all my pictures and videos from the 1D and 5sos concert,and from Vet Camp #hatinglife #sobbing"
4,7925,Been fighting iTunes all night! I only want the music I $&@*# paid for


In [282]:
def get_url(text):
    u1 = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',text)
    #text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)
    
#     u2 = re.findall('[^ ]+\.[com]','',text)
#     text = re.sub('[^ ]+\.[com]','',text)
    return len(u1)
def count_hash(text):
    text = "".join(text)
    u = re.findall('#[a-zA-Z0-9]+',text)
    return len(u)



def clear_text(text):
    def decontracted(phrase):
       # specific
        phrase = re.sub(r"won\'t", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        return phrase

    text = decontracted(text)
    
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)
    text = re.sub('[^a-zA-Z]+',' ',text)
    text = [t for t in text.split() if len(t)>2]
    text = " ".join(text)

    
    
    return text

def html_decoding(text):
    # &amp etc etc
    text = BeautifulSoup(text, 'lxml')
    text = text.get_text()
    return text


train['tweet'] = train['tweet'].apply(lambda x: html_decoding(x))  
train["urls"] = train["tweet"].apply(lambda x: get_url(x))
train["n_hash"] = train["tweet"].apply(lambda x: count_hash(x))
train['tweet'] = train['tweet'].apply(lambda x: clear_text(x))


train["word_count"] = train["tweet"].apply(lambda x: len([t for t in x.split() if len(t)>1]))
train["stop_count"] = train["tweet"].apply(lambda x: len([t for t in x.split() if t in stopwords_]))
train["tweet"] = train["tweet"].apply(lambda x: x.lower())

In [283]:
test['tweet'] = test['tweet'].apply(lambda x: html_decoding(x))  
test["urls"] = test["tweet"].apply(lambda x: get_url(x))
test["n_hash"] = test["tweet"].apply(lambda x: count_hash(x))
test['tweet'] = test['tweet'].apply(lambda x: clear_text(x))


test["word_count"] = test["tweet"].apply(lambda x: len([t for t in x.split() if len(t)>1]))
test["stop_count"] = test["tweet"].apply(lambda x: len([t for t in x.split() if t in stopwords_]))
test["tweet"] = test["tweet"].apply(lambda x: x.lower())

In [284]:
train.head()

,id,label,tweet,urls,n_hash,word_count,stop_count
0,1,0,fingerprint pregnancy test android apps beautiful cute health igers iphoneonly iphonesia iphone,1,11,12,0
1,2,0,finally transparant silicon case thanks uncle yay sony xperia sonyexperias,1,5,10,0
2,3,0,love this would you talk makememories unplug relax iphone smartphone wifi connect,1,8,12,2
3,4,0,wired know george was made that way iphone cute daventry home,1,4,11,2
4,5,1,what amazing service apple will not even talk about question have unless pay them for their stupid support,0,0,18,7


In [285]:
test.head()

,id,tweet,urls,n_hash,word_count,stop_count
0,7921,hate the new iphone upgrade not let download apps ugh apple sucks,0,3,12,2
1,7922,currently shitting fucking pants apple imac cashmoney raddest swagswagswag,1,5,9,0
2,7923,would like puts some roms ipad that possible yes but would not that block the screen,0,0,16,6
3,7924,ipod officially dead lost all pictures and videos from the and sos concert and from vet camp hatinglife sobbing,0,2,19,7
4,7925,been fighting itunes all night only want the music paid for,0,0,11,4


In [262]:
train.groupby(["label"])["n_hash","word_count","stop_count"].describe()

n_hash                                               word_count  \
        count      mean       std  min  25%  50%  75%   max      count   
label                                                                    
0      5894.0  7.318799  4.850908  0.0  4.0  7.0  9.0  33.0  5894.0      
1      2026.0  1.788746  2.095920  0.0  0.0  1.0  2.0  26.0  2026.0      

                                                        stop_count            \
            mean       std  min   25%   50%   75%   max      count      mean   
label                                                                          
0      13.612657  5.629380  1.0  10.0  12.0  15.0  41.0  5894.0     1.256023   
1      13.624877  4.024917  4.0  11.0  13.0  16.0  46.0  2026.0     3.433860   

                                           
            std  min  25%  50%  75%   max  
label                                      
0      1.692893  0.0  0.0  1.0  2.0  15.0  
1      2.221515  0.0  2.0  3.0  5.0  21.0

In [299]:
train_tweets = ' '
for t in train.tweet:
    train_tweets += t + " "
    
test_tweets = ' '
for t in test.tweet:
    test_tweets += t + " "
    
tweets = train_tweets + " " + test_tweets

tweets =  [tweets]

cv = CountVectorizer(ngram_range=(1,1),max_features=2500,stop_words=stopwords_)
cv.fit(tweets)

train_corpus = []
for t in train.tweet:
    train_corpus.append(t)
    
test_corpus = []
for t in test.tweet:
    test_corpus.append(t)
    

X = cv.transform(train_corpus).toarray()
y = train.label
X_test = cv.transform(test_corpus).toarray()


clfs = [DecisionTreeClassifier(),GaussianNB(),RandomForestClassifier(),LogisticRegression()]
for clf in clfs:
    n = 10
    skf = StratifiedKFold(n_splits=n,shuffle=True,random_state=101)
    cv10 = list(skf.split(X,y))
    cv_accuracy=[]
    cv_roc_auc=[]
    cv_f1=[]
    cv_precison=[]
    cv_recall=[]

    for i in range(n):
        #train indices
        X_train = []
        for idx in cv10[i][0]:
            X_train.append(X[idx])
        y_train = y[cv10[i][0]]

        #validation indices
        X_val = []
        for idx in cv10[i][1]:
            X_val.append(X[idx])
        y_val = y[cv10[i][1]]

        clf.fit(X_train,y_train)
        predicted = clf.predict(X_val)
        predicted_proba = clf.predict_proba(X_val)[:,1]
        
        accuracy = accuracy_score(y_val,predicted)
        precision = precision_score(y_val,predicted)
        f1 = f1_score(y_val,predicted)
        recall = recall_score(y_val,predicted)
        roc_auc = roc_auc_score(y_val,predicted_proba)
        
        #print(round(score,3),end=", ")
        cv_accuracy.append(accuracy) 
        cv_precison.append(precision)
        cv_f1.append(f1)
        cv_recall.append(recall)
        cv_roc_auc.append(roc_auc)
    
    print(clf)
    print("mean_accuracy     : ",round(np.mean(cv_score),2))
    print("\nmean_precision  : ",round(np.mean(cv_precison),2))
    print("\nmean_f1.        : ",round(np.mean(cv_f1),2))
    print("\nmean_reacall    : ",round(np.mean(cv_recall),2))
    print("\nmean_roc_auc    : ",round(np.mean(cv_roc_auc),2))
    print("\n")

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
mean_accuracy     :  0.88

mean_precision  :  0.7

mean_f1.        :  0.69

mean_reacall    :  0.69

mean_roc_auc    :  0.79


GaussianNB(priors=None, var_smoothing=1e-09)
mean_accuracy     :  0.88

mean_precision  :  0.5

mean_f1.        :  0.63

mean_reacall    :  0.84

mean_roc_auc    :  0.78


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,


In [304]:
# tfidf vectorizer  with svc
train_tweets = ' '
for t in train.tweet:
    train_tweets += t + " "

    
test_tweets = ' '
for t in test.tweet:
    test_tweets += t + " "
    
tweets = train_tweets + " " + test_tweets

tweets =  [train_tweets]

train_corpus = []
for t in train.tweet:
    train_corpus.append(t)
    
test_corpus = []
for t in test.tweet:
    test_corpus.append(t)
    
tfidf_vec = TfidfVectorizer()
tfidf_vec.fit(tweets)

X = tfidf_vec.transform(train_corpus).toarray()
y = train.label
X_test = tfidf_vec.transform(test_corpus).toarray()



n = 10
skf = StratifiedKFold(n_splits=n,shuffle=True,random_state=101)
cv10 = list(skf.split(X,y))
cv_score=[]
test_pred = np.zeros((len(X_test),1))

for i in range(n):
    #train indices
    X_train = []
    for idx in cv10[i][0]:
        X_train.append(X[idx])
    y_train = y[cv10[i][0]]

    #validation indices
    X_val = []
    for idx in cv10[i][1]:
        X_val.append(X[idx])
    y_val = y[cv10[i][1]]

#     lr = LogisticRegression()
#     lr.fit(X_train,y_train)
    svc  = LinearSVC(C=1)
    svc.fit(X_train,y_train)
    score = accuracy_score(y_val,svc.predict(X_val))
    print(round(score,3),end=", ")
    cv_score.append(score)  

    #pred = svc.predict_proba(X_test)[:,1].reshape(-1,1)
    test_pred += pred
print("\nmean_score: ",round(np.mean(cv_score),2))
test_pred = test_pred/10

temp = test.copy()
temp["prob"] = test_pred
temp.head()
temp["label"] = np.where(temp['prob']>=0.5,1,0)
temp.index = temp.id
temp = temp[['label']]
temp.to_csv('try4.csv')

0.892, 0.881, 0.88, 0.866, 0.898, 0.885, 0.88, 0.901, 0.891, 0.9, 
mean_score:  0.89
